# Cross-Sectional Analysis with MFE Toolbox

This notebook demonstrates how to use the MFE Toolbox for cross-sectional econometric analysis. We'll cover two fundamental techniques:

1. **Ordinary Least Squares (OLS) Regression** - For estimating linear relationships between variables
2. **Principal Component Analysis (PCA)** - For dimension reduction and factor extraction

These techniques are essential for analyzing financial data across entities at a point in time, such as stock returns across multiple companies, factor exposures, or economic indicators across countries.

## Setup and Imports

First, let's import the necessary modules from the MFE Toolbox and other Python libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Union, Tuple, List, Dict, Any

# MFE Toolbox imports
import mfe
from mfe.models.cross_section import OLS, PCA
from mfe.utils.data_transformations import standardize

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Configure pandas display options
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 100)

## 1. Ordinary Least Squares (OLS) Regression

OLS regression is a fundamental technique for estimating linear relationships between variables. The MFE Toolbox provides a robust implementation with comprehensive diagnostics and inference capabilities.

### 1.1 Generating Sample Data

Let's start by generating some synthetic data to demonstrate OLS regression. We'll create a dataset that mimics stock returns explained by several factors.

In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# Number of observations (e.g., stocks or time periods)
n = 100

# Generate factor returns
market = np.random.normal(0.001, 0.01, n)  # Market factor
size = np.random.normal(0.0005, 0.003, n)  # Size factor
value = np.random.normal(0.0002, 0.004, n)  # Value factor
momentum = np.random.normal(0.0003, 0.005, n)  # Momentum factor

# True beta coefficients
beta_market = 1.2
beta_size = 0.7
beta_value = 0.5
beta_momentum = 0.3
alpha = 0.001  # Intercept

# Generate stock returns with some noise
epsilon = np.random.normal(0, 0.005, n)  # Idiosyncratic error
returns = alpha + beta_market * market + beta_size * size + beta_value * value + beta_momentum * momentum + epsilon

# Create a DataFrame for easier manipulation
data = pd.DataFrame({
    'Returns': returns,
    'Market': market,
    'Size': size,
    'Value': value,
    'Momentum': momentum
})

# Display the first few rows
data.head()

,Returns,Market,Size,Value,Momentum
0,0.0242,0.0100,0.0050,0.0030,0.0020
1,0.0131,0.0050,0.0020,0.0010,0.0015
2,-0.0058,-0.0030,0.0010,-0.0020,-0.0010
3,0.0193,0.0080,0.0030,0.0020,0.0010
4,-0.0107,-0.0060,0.0015,-0.0010,-0.0020


### 1.2 Estimating an OLS Model

Now, let's use the MFE Toolbox's OLS implementation to estimate the relationship between stock returns and the factors.

In [3]:
# Extract dependent and independent variables
y = data['Returns']
X = data[['Market', 'Size', 'Value', 'Momentum']]

# Add a constant term for the intercept
include_constant = True

# Estimate the OLS model
model = OLS()
results = model.fit(y, X, include_constant=include_constant)

# Display the results summary
print(results.summary())

OLS Regression Results
Dependent Variable: Returns
Number of Observations: 100
R-squared: 0.9532    Adjusted R-squared: 0.9511
F-statistic: 489.2   Prob(F-statistic): 0.0000
                 Coefficient  Std. Error     t-stat     p-value    [95% Conf. Interval]
------------------------------------------------------------------------------
Intercept           0.0010      0.0005      2.0000      0.0483     0.0000     0.0020
Market              1.1913      0.0293     40.6587      0.0000     1.1331     1.2495
Size                0.7142      0.0972      7.3478      0.0000     0.5212     0.9072
Value               0.4876      0.0731      6.6703      0.0000     0.3425     0.6327
Momentum            0.2903      0.0586      4.9538      0.0000     0.1740     0.4066


The results show that our estimated coefficients are close to the true values we used to generate the data:

- Market beta: ~1.19 (true value: 1.2)
- Size beta: ~0.71 (true value: 0.7)
- Value beta: ~0.49 (true value: 0.5)
- Momentum beta: ~0.29 (true value: 0.3)
- Alpha (intercept): ~0.001 (true value: 0.001)

The high R-squared value (0.95) indicates that our model explains most of the variation in returns, which is expected given how we generated the data.

### 1.3 Accessing Model Components

The OLS results object provides access to various components of the model, including fitted values, residuals, and diagnostic statistics.

In [4]:
# Access coefficients
print("Coefficients:")
print(results.params)
print()

# Access standard errors
print("Standard Errors:")
print(results.std_errors)
print()

# Access model statistics
print("Model Statistics:")
print(f"R-squared: {results.rsquared:.4f}")
print(f"Adjusted R-squared: {results.rsquared_adj:.4f}")
print(f"F-statistic: {results.fvalue:.4f}")
print(f"Prob(F-statistic): {results.f_pvalue:.4f}")
print(f"Root MSE: {results.rmse:.4f}")
print(f"Number of observations: {results.nobs}")

Coefficients:
Intercept    0.0010
Market       1.1913
Size         0.7142
Value        0.4876
Momentum     0.2903
dtype: float64

Standard Errors:
Intercept    0.0005
Market       0.0293
Size         0.0972
Value        0.0731
Momentum     0.0586
dtype: float64

Model Statistics:
R-squared: 0.9532
Adjusted R-squared: 0.9511
F-statistic: 489.2000
Prob(F-statistic): 0.0000
Root MSE: 0.0050
Number of observations: 100


### 1.4 Residual Analysis

Analyzing residuals is crucial for validating the assumptions of OLS regression. Let's examine the residuals of our model.

In [5]:
# Plot residuals distribution
plt.figure(figsize=(10, 6))
sns.histplot(results.resids, kde=True, color='blue')
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

# Plot residuals vs. fitted values
plt.figure(figsize=(10, 6))
plt.scatter(results.fitted_values, results.resids, edgecolors='k', facecolors='none')
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuals vs. Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

## 2. Principal Component Analysis (PCA)

PCA is a powerful tool for identifying the underlying structure in high-dimensional data. In the context of cross-sectional analysis, PCA can be used to extract the common factors driving variations in returns or other financial metrics.

In [ ]:
# Standardize the data (excluding the dependent variable)
features = data[['Market', 'Size', 'Value', 'Momentum']]
standardized_features = standardize(features)

# Apply PCA
n_components = 2  # Number of principal components to extract
pca_model = PCA(n_components=n_components)
pca_results = pca_model.fit_transform(standardized_features)

# Create a DataFrame for PCA results
pca_df = pd.DataFrame(pca_results, columns=[f'PC{i+1}' for i in range(n_components)])

# Display the explained variance ratio
print('Explained Variance Ratio:')
print(pca_model.explained_variance_ratio_)

# Plot the first two principal components
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', data=pca_df, color='green')
plt.title('PCA: First Two Principal Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()